If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
!pip install datasets
!pip install transformers
!pip install accelerate -U
# Transformers installation
# To install from source instead of the last release, comment the command above and uncomment the following one.
! pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-8qt_lse5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-8qt_lse5
  Resolved https://github.com/huggingface/transformers.git to commit b109257f4fb8b1166e7c53cc5418632014ed53a5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
import transformers

print(transformers.__version__)

4.40.0.dev0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](images/causal_language_modeling.png)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](images/masked_language_modeling.png)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [CyberNative]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [6]:
from datasets import load_dataset
dataset = load_dataset("CyberNative/github_cybersecurity_READMEs")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [7]:
dataset["train"] = load_dataset("CyberNative/github_cybersecurity_READMEs", split='train[:80%]')
dataset["validation"] = load_dataset("CyberNative/github_cybersecurity_READMEs", split='train[80%:90%]')
dataset["test"] = load_dataset("CyberNative/github_cybersecurity_READMEs", split='train[90%:]')

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [8]:
dataset["train"][10]

{'text': "# Devel\n\nURL: https://app.hackthebox.com/machines/Devel\n\nLevel: Easy\n\nDate 31 May 2020\n\n## Walkthrough\n\n- [Enumeration](#enumeration)\n- [Privesc](#privesc)\n\n# Enumeration\n\n## NMAP\n\n```\n# Nmap 7.91 scan initiated Sat Apr  3 11:13:46 2021 as: nmap -Pn -p- -T4 -oN nmap.txt 10.10.10.5\nNmap scan report for 10.10.10.5\nHost is up (0.044s latency).\nNot shown: 65533 filtered ports\nPORT   STATE SERVICE\n21/tcp open  ftp\n80/tcp open  http\n\n# Nmap done at Sat Apr  3 11:15:16 2021 -- 1 IP address (1 host up) scanned in 90.09 seconds\n```\n\n```\n# Nmap 7.80 scan initiated Sun May 31 17:52:00 2020 as: nmap -A -T4 -p- -oN nmap.txt 10.10.10.5\nNmap scan report for 10.10.10.5\nHost is up (0.048s latency).\nNot shown: 65533 filtered ports\nPORT   STATE SERVICE VERSION\n21/tcp open  ftp     Microsoft ftpd\n| ftp-anon: Anonymous FTP login allowed (FTP code 230)\n| 03-18-17  02:06AM       <DIR>          aspnet_client\n| 06-03-20  06:06PM                 2758 file.aspx\n| 

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(dataset["train"])

As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

# **Preprocess**

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/206 [00:00<?, ? examples/s]

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

In [15]:
model_checkpoint = "distilgpt2"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [17]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [18]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [19]:
tokenized_datasets["train"][1]

{'input_ids': [198,
  2,
  4600,
  707,
  5927,
  12,
  6057,
  12,
  12961,
  63,
  198,
  198,
  58,
  0,
  58,
  38,
  270,
  16066,
  5964,
  16151,
  5450,
  1378,
  9600,
  13,
  26662,
  82,
  13,
  952,
  14,
  12567,
  14,
  43085,
  14,
  39870,
  71,
  4309,
  1495,
  14,
  707,
  5927,
  12,
  6057,
  12,
  12961,
  15437,
  7,
  5450,
  1378,
  12567,
  13,
  785,
  14,
  39870,
  71,
  4309,
  1495,
  14,
  707,
  5927,
  12,
  6057,
  12,
  12961,
  14,
  2436,
  672,
  14,
  12417,
  14,
  43,
  2149,
  24290,
  8,
  198,
  198,
  15506,
  63,
  198,
  1532,
  345,
  1064,
  326,
  617,
  6117,
  389,
  407,
  1762,
  11,
  345,
  460,
  2391,
  6330,
  262,
  20579,
  351,
  308,
  76,
  71,
  4309,
  1495,
  13,
  220,
  198,
  5574,
  345,
  460,
  3758,
  281,
  2071,
  329,
  502,
  13,
  198,
  15506,
  63,
  198,
  198,
  2235,
  1374,
  284,
  8676,
  30,
  198,
  12,
  3740,
  1378,
  12567,
  13,
  785,
  14,
  21217,
  403,
  34,
  8635,
  14,
  3642,
  4163,

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [20]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [21]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [22]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [23]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'.io/github/license/harisqazi1/Cybersecurity)](https://github.com/harisqazi1/Cybersecurity/blob/main/LICENSE)\n\n</p>\n\n# Cybersecurity Document\nThis document is meant to be a Swiss Army Knife for entry level Cybersecurity jobs and to learn hacking skills. A work in progress, so if you see mistakes, please mention it in the "Issues" section. \n\nNOTE: I DO NOT OWN ANY OF THIS INFORMATION. THIS IS JUST MEANT TO BE A COMPILATION OF VARIOUS'

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [24]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

And some `TrainingArguments`:

In [25]:
from transformers import TrainingArguments

In [26]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-cybersecurity_readme",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

In [27]:
# !pip install transformers
from transformers.trainer import Trainer

print(Trainer.__init__.__doc__)

None


In [28]:
small_train_dataset = lm_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = lm_datasets["validation"].shuffle(seed=42).select(range(1000))

We pass along all of those to the `Trainer` class:

In [29]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=lm_datasets["train"],
#     eval_dataset=lm_datasets["validation"],
# )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

And we can train our model:

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,3.033010
2,No log,2.990992
3,No log,2.986135


TrainOutput(global_step=375, training_loss=3.3700319010416666, metrics={'train_runtime': 689.4567, 'train_samples_per_second': 4.351, 'train_steps_per_second': 0.544, 'total_flos': 97986281472000.0, 'train_loss': 3.3700319010416666, 'epoch': 3.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [31]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 19.81


You can now upload the result of the training to the Hub, just execute this instruction:

In [32]:
trainer.push_to_hub()

events.out.tfevents.1713098246.cb6a4e3c1638.14960.0:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

events.out.tfevents.1713098506.cb6a4e3c1638.14960.1:   0%|          | 0.00/10.0k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

events.out.tfevents.1713099596.cb6a4e3c1638.20503.0:   0%|          | 0.00/6.22k [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

events.out.tfevents.1713098831.cb6a4e3c1638.14960.2:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

events.out.tfevents.1713100367.cb6a4e3c1638.20503.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LDDon/distilgpt2-finetuned-cybersecurity_readme/commit/0c28e48a50dffebc2c15ce1ba6bb665d78251966', commit_message='End of training', commit_description='', oid='0c28e48a50dffebc2c15ce1ba6bb665d78251966', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

We will use the [`distilroberta-base`](https://huggingface.co/distilroberta-base) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead:

In [33]:
model_checkpoint = "distilroberta-base"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [34]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [35]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [36]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We redefine our `TrainingArguments`:

In [37]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-cybersecurity_readme2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

Like before, the last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-wikitext2"` or `"huggingface/bert-finetuned-wikitext2"`).

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [38]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [39]:
small_train_dataset = lm_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = lm_datasets["validation"].shuffle(seed=42).select(range(1000))

Then we just have to pass everything to `Trainer` and begin training:

In [40]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
)

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.815138
2,No log,1.752780
3,No log,1.701787


TrainOutput(global_step=375, training_loss=2.1721022135416668, metrics={'train_runtime': 545.9654, 'train_samples_per_second': 5.495, 'train_steps_per_second': 0.687, 'total_flos': 99466354944000.0, 'train_loss': 2.1721022135416668, 'epoch': 3.0})

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [42]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 5.64


You can now upload the result of the training to the Hub, just execute this instruction:

In [43]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1713101175.cb6a4e3c1638.20503.3:   0%|          | 0.00/359 [00:00<?, ?B/s]

events.out.tfevents.1713100540.cb6a4e3c1638.20503.2:   0%|          | 0.00/5.91k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LDDon/distilroberta-base-finetuned-cybersecurity_readme2/commit/16e5bb2bd8f78d9270ffb6f81da9f31cd20a2bc4', commit_message='End of training', commit_description='', oid='16e5bb2bd8f78d9270ffb6f81da9f31cd20a2bc4', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("sgugger/my-awesome-model")
```